#### Notebook Content: 

* Read CSV files and prepare data for exploratory data analysis

#### Files used:
* Users data,
* Transactions,
* Notification - all the notifications sent to users and
* Devices

#### Load libraries

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [21]:
# Set working directory
import os
os.getcwd() # get working directory 
os.chdir('/Users/vinaykumar/Documents/Python_scripts/Project A/Data/') 

#### Read users, transactions,notification and devices data from csv files

In [22]:
users_df = pd.read_csv('users.csv',header=0) #users data
users_df = users_df.drop(['Unnamed: 0'], axis = 1)
users_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19430 entries, 0 to 19429
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   user_id                   19430 non-null  object 
 1   birth_year                19430 non-null  int64  
 2   country                   19430 non-null  object 
 3   created_date              19430 non-null  object 
 4   num_contacts              19430 non-null  int64  
 5   num_successful_referrals  19430 non-null  int64  
 6   engaged                   12087 non-null  float64
dtypes: float64(1), int64(3), object(3)
memory usage: 1.0+ MB


In [24]:
transactions = pd.read_csv('transactions.csv',header=0) # transactions data
transactions = transactions.drop(['Unnamed: 0'], axis = 1) # drop unwanted column
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82202 entries, 0 to 82201
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   transaction_id     82202 non-null  object 
 1   transactions_type  82202 non-null  object 
 2   amount_usd         82202 non-null  float64
 3   user_id            82202 non-null  object 
 4   created_date       82202 non-null  object 
dtypes: float64(1), object(4)
memory usage: 3.1+ MB


In [5]:
notifications = pd.read_csv('notifications.csv',header=0) # notification data
notifications.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121813 entries, 0 to 121812
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   reason        121813 non-null  object
 1   channel       121813 non-null  object
 2   status        121813 non-null  object
 3   user_id       121813 non-null  object
 4   created_date  121813 non-null  object
dtypes: object(5)
memory usage: 4.6+ MB


In [25]:
devices = pd.read_csv('devices.csv',header=0) # devices data
devices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19430 entries, 0 to 19429
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   brand    19430 non-null  object
 1   user_id  19430 non-null  object
dtypes: object(2)
memory usage: 303.7+ KB


In [26]:
# create a copy of user dataframe 
users = users_df.copy()

##### Rename calumns in users and notification dataframes

In [27]:
users = users.rename(columns = {'created_date':'account_date'}) 

In [28]:
notifications = notifications.rename(columns = {'created_date':'notification_date'})

##### Convert Date columns from object to date format

In [29]:
# convert account_date
users['account_date'] = pd.to_datetime(users.account_date,format ='%Y-%m-%d')

In [30]:
# convert notification_date
notifications['notification_date'] = pd.to_datetime(notifications.notification_date,format ='%Y-%m-%d')

##### New Feature calculation in users data - Age of Customer and Age of Account

In [31]:
# Use birth_year column to calculate age of customer 
users['customer_age'] =  2020 - users['birth_year']
users =  users.drop('birth_year', axis=1) # drop birth_year column


In [32]:
# use account_date column to calculate age of account
users['account_age_months'] =  round(pd.to_numeric((pd.to_datetime('2020-01-31')- users['account_date'])/np.timedelta64(1,'M')))
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19430 entries, 0 to 19429
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   user_id                   19430 non-null  object        
 1   country                   19430 non-null  object        
 2   account_date              19430 non-null  datetime64[ns]
 3   num_contacts              19430 non-null  int64         
 4   num_successful_referrals  19430 non-null  int64         
 5   engaged                   12087 non-null  float64       
 6   customer_age              19430 non-null  int64         
 7   account_age_months        19430 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(2)
memory usage: 1.2+ MB


###### Calculate number of notification sent to every customer by counting all the notifications sent to per customers

In [33]:
notifications1 = notifications.groupby('user_id').agg({'notification_date': 'count'})
notifications1.columns = ['notifications_count']

##### Merge notification count feature to users dataframe 

In [34]:
users = users.merge(notifications1, on=['user_id'], how='left') 

##### Merge device feature to users dataframe 

In [35]:
### merge devices info
users =  users.merge(devices, on=['user_id'],how='left')

In [36]:
# Check final data set
users.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19430 entries, 0 to 19429
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   user_id                   19430 non-null  object        
 1   country                   19430 non-null  object        
 2   account_date              19430 non-null  datetime64[ns]
 3   num_contacts              19430 non-null  int64         
 4   num_successful_referrals  19430 non-null  int64         
 5   engaged                   12087 non-null  float64       
 6   customer_age              19430 non-null  int64         
 7   account_age_months        19430 non-null  float64       
 8   notifications_count       18953 non-null  float64       
 9   brand                     19430 non-null  object        
dtypes: datetime64[ns](1), float64(3), int64(3), object(3)
memory usage: 1.6+ MB


In [37]:
users = users.fillna(0)
users.head()

,user_id,country,account_date,num_contacts,num_successful_referrals,engaged,customer_age,account_age_months,notifications_count,brand
0,user_0,PL,2018-01-13 05:15:15.599466,3,0,1.0,31,25.0,8.0,Apple
1,user_1,GB,2018-01-29 03:38:46.676876,21,0,1.0,45,24.0,7.0,Apple
2,user_2,PL,2018-01-18 19:17:31.229096,21,0,1.0,33,24.0,12.0,Android
3,user_3,FR,2018-01-15 18:47:56.723104,0,0,1.0,26,24.0,8.0,Apple
4,user_4,GB,2018-01-11 00:36:46.673673,2,0,1.0,35,25.0,4.0,Apple
